# MNIST test accuracy    SVM: 92.450%  LR: 91.230%
# MNIST first 16 pcomp test accuracy    SVM: 88.335%  LR: 82.710%
# MNIST first 256 pcomp test accuracy    SVM: 92.795%  LR: 90.850%

In [28]:
import sys
import os
import time
import numpy as np
import theano
import theano.tensor as T
import lasagne
import sklearn
from sklearn.linear_model import LogisticRegression as LR


In [2]:
from mnist import *

In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

In [9]:
print(X_train.shape, X_val.shape, X_test.shape)

(50000, 1, 28, 28) (10000, 1, 28, 28) (10000, 1, 28, 28)


In [7]:
y_train.shape

(50000,)

# CNN baseline: 99%

In [4]:
num_conv = 32
mid_neurons = 256

In [5]:
# main(model='cnn')

In [6]:
input_var = T.tensor4('inputs')
target_var = T.ivector('targets')
network = build_cnn(input_var, num_conv, mid_neurons)
feature_layer = lasagne.layers.get_all_layers(network)[-2]
feature = lasagne.layers.get_output(feature_layer, deterministic=True)
feature_fn = theano.function([input_var], feature)

In [26]:
train_data = np.zeros((50000,mid_neurons))
test_data = np.zeros((10000,mid_neurons))

i = 0
for batch in iterate_minibatches(X_train, y_train, 500, shuffle=False):
        inputs, targets = batch
        out = feature_fn(inputs)
        train_data[i*500:(i+1)*500,:] = out
        i += 1
i = 0
for batch in iterate_minibatches(X_test, y_test, 500, shuffle=False):
        inputs, targets = batch
        out = feature_fn(inputs)
        test_data[i*500:(i+1)*500,:] = out
        i += 1

In [31]:
def train_and_eval( model, train_x, train_y, test_x, test_y ):
    model.fit( train_x, train_y )
    p = model.predict( test_x )
    OA = sum(test_y==p)/len(test_y)
    return OA

In [34]:
y_train.shape

(50000,)

In [35]:
lr = LR(C=1)
auc = train_and_eval( lr, train_data, y_train, \
    test_data, y_test )
auc

0.95630000000000004

In [36]:
svm=sklearn.svm.SVC(kernel='linear',C=1,shrinking=False)
auc = train_and_eval( svm, train_data, y_train, \
    test_data, y_test )
auc

0.96130000000000004

In [21]:
inputs = X_val[:500,:,:,:]
out = feature_fn(inputs)

In [22]:
out.shape

(500, 256)